# 02 models

Træn & sammenlign fasttext med sklearn på samme dataset og se hvilken model der er mest præcis.

### TODO:

- lav træningenfilen til en variabel. så vi kan bruge forskellige csv filer til træning
- oprydning
- QOL barplot på resultat

### Installs:

In [1]:
# Funktion der kan bruges til at installere manglende packages
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Installér fasttext
install('tabulate')

### imports:

In [2]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import fasttext

from tabulate import tabulate

### Train Sklearn model:

In [3]:
df = pd.read_csv("./data/sklearn.csv")
df.head()

#Term Frequency — Inverse Document Frequency
tfidf = TfidfVectorizer()

x = df['review']
y = df['rating']

x = tfidf.fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state=5)

clf=SVC(kernel='linear',probability=True, random_state=32, decision_function_shape='ovr',max_iter=10000)
clf.fit(x_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=10000, probability=True, random_state=32, shrinking=True,
    tol=0.001, verbose=False)

### Train FastText model:

In [14]:
#model = fasttext.train_supervised(input="./data/sklearn.csv", epoch=32, lr=0.5, wordNgrams=2, bucket=200000, dim=30, loss='ova')
model = fasttext.load_model('./data/fasttext_trained_model.bin')

### Model comparison:

In [19]:
z = df['review']

sklearnList = []
fasttextList = []
for line in z:
    fasttextList.append(model.predict(line))
    vec1 = tfidf.transform([line])
    sklearnList.append(clf.predict(vec1))

labelToTest = '__label__5 ' 

selectLabels = [labelToTest]
label = df[df['rating'].isin(selectLabels)]
# print('Total amount of reviews that gotten', labelToTest,len(label))

sk = list(filter(lambda k: labelToTest in k, sklearnList))
# print('Total amount of ', labelToTest + 'according to sklearn' , len(sk))

fasttextLabels = []
for index in fasttextList:
    rep = str(index[0])
    l = rep.split("'")
    l = l[1]+" "
    fasttextLabels.append(l)

ft = list(filter(lambda k: labelToTest in k, fasttextLabels))
# print('Total amount of ', labelToTest + 'according to fasttext' , len(ft))

def calc_accuracy(data):
    if(data > 100):
        data = 100 - data + 100
        return data
    else: 
        data = data - 100 + 100
        return data      


hundredp = len(label)

roundft = len(ft)/hundredp*100
roundsk = len(sk)/hundredp*100


roundft = calc_accuracy(roundft)
roundsk = calc_accuracy(roundsk)    

# print('Accurary in % for fasttext' ,round(roundft,2),'%')
# print('Accurary in % for sklearn' ,round(roundsk,2),"%")
table = [['stars ','total reviews ', 'sklearn predict',' sklearn accuracy', 'fastText predict', ' fasttext accuracy' ], [labelToTest[-2] , len(label), len(sk),(round(roundsk,2)), len(ft), round(roundft,2) ]]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))


╒══════════╤══════════════════╤═══════════════════╤═════════════════════╤════════════════════╤══════════════════════╕
│   stars  │   total reviews  │   sklearn predict │    sklearn accuracy │   fastText predict │    fasttext accuracy │
╞══════════╪══════════════════╪═══════════════════╪═════════════════════╪════════════════════╪══════════════════════╡
│        5 │             1879 │              2035 │                91.7 │               1918 │                97.92 │
╘══════════╧══════════════════╧═══════════════════╧═════════════════════╧════════════════════╧══════════════════════╛
